<a href="https://colab.research.google.com/github/cluePrints/fastai-v3-notes/blob/master/fastai3_part2_09c_progress_bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Env state dump

This is going to be useful for troubleshooting later

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html -q

In [3]:
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==2.4.1
astor==0.7.1
astropy==3.0.5
atari-py==0.1.7
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.6
autograd==1.2
Babel==2.6.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
bokeh==1.0.4
boto==2.49.0
boto3==1.9.130
botocore==1.12.130
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.0
certifi==2019.3.9
cffi==1.12.2
chainer==5.0.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cupy-cuda100==5.2.0
cvxopt==1.2.3
cvxpy==1.0.15
cycler==0.10.0
cymem==2.0.2
Cython==0.29.6
cytoolz==0.9.0.1
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.5.0
dill==0.2.9
distributed==1.25.3
Django==2.2
dlib==19.16.0
dm-sonnet==1.23
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
e

# Imports

In [0]:
# download fast ai exports
!curl -s -L https://api.github.com/repos/fastai/fastai_docs/tarball/02697011d061d82b7922a650fc494915f4926c96 | tar xz --wildcards "*/dev_course/dl2/exp/*.py" --strip-components=3

In [0]:
from exp.nb_09b import *

# Body

In [0]:
import time
from fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

In [0]:
import numpy as np
def my_collate(batch):
  from torch.utils.data.dataloader import default_collate
  xb, yb = default_collate(batch)
  return xb, yb

In [0]:
from collections import defaultdict
class Numericalizer():
  def __init__(self):
    self.max_index = -1
    self.original_to_index = defaultdict(self._next_index)
    self.index_to_original = defaultdict()
    
  def __call__(self, original):
    index = self.original_to_index[original]
    self.index_to_original[index] = original
    return index
    
  def _next_index(self):
    self.max_index += 1
    return self.max_index
  
n = Numericalizer()


test_eq(n('aa'), 0)
test_eq(n('a'), 1)
test_eq(n('b'), 2)
test_eq(n.original_to_index['a'], 1)
test_eq(n.original_to_index['b'], 2)
test_eq(n.index_to_original[1], 'a')
test_eq(n.index_to_original[2], 'b')
test_eq(n('a'), 1)

In [0]:
parent_and_numericalize = lambda x: label_numericalizer(parent_labeler(x))

In [0]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
batch_size = 64
label_numericalizer = Numericalizer()

image_list = ImageList.from_files(path, tfms=tfms)
split_list = SplitData.split_by_func(image_list, partial(grandparent_splitter, valid_name='val'))
labeled_list = label_by_func(split_list, parent_and_numericalize)
data = labeled_list.to_databunch(batch_size, c_in=3, c_out=10, num_workers=0, collate_fn=my_collate)
assert data.train_dl.num_workers == 0, f"Got {data.train_dl.num_workers}"
assert data.valid_dl.num_workers == 0, f"Got {data.valid_dl.num_workers}"

In [0]:
master_bar.on_iter_end??

In [0]:
import pdb; 
class ProgressCallback(Callback):
  def begin_fit(self):
    self.epochs_bar = master_bar(range(self.run.epochs))
    self.epochs_bar.on_iter_begin()

  def begin_epoch(self):    self._set_new_batches_bar()
  def begin_validate(self): self._set_new_batches_bar()

  def begin_batch(self): self._update_batches_progress()
  def after_batch(self): self._update_batches_progress()

  def after_fit(self):
    if hasattr(self, 'epochs_bar'): self.epochs_bar.on_iter_end()
    
  def _update_batches_progress(self):
    self.batches_bar.update(self.run.n_iter)
    
  def _set_new_batches_bar(self):
    self.run.n_iter = 0
    self.epochs_bar.update(int(self.n_epochs))
    self.batches_bar = progress_bar(self.dl, parent=self.epochs_bar, auto_update=False)

In [0]:
callback_funcs = [partial(AvgStatsCallback,accuracy),
        CudaCallback,
        ProgressCallback,
        partial(BatchTransformXCallback, norm_imagenette)]

In [0]:
model_func = get_model_func()
loss_func = F.cross_entropy
num_filters = [32]*4
layer_func = conv_layer
lr = 0.1
learner = get_learner(num_filters, data, lr, layer_func, loss_func, cb_funcs=callback_funcs)

In [15]:
learner.fit(2)

train: [1.994650794458663, tensor(0.2926, device='cuda:0')]
valid: [1.8051944580078125, tensor(0.3700, device='cuda:0')]
train: [1.636410800323794, tensor(0.4405, device='cuda:0')]
valid: [1.504024169921875, tensor(0.4660, device='cuda:0')]


In [16]:
class ProgressCallbackV2(ProgressCallback):
  def begin_fit(self):
    super().begin_fit()
    self.old_logger = self.run.logger
    self.run.logger = self._log

    # state is shameful, so is dependency betwee two callbacks this sorta relaxes!
    self.logged_header = False
    self.last_train_event = False
    self.last_valid_event = False

  def _log(self, log_event):
    if not isinstance(log_event, AvgStats):
      self.old_logger(log_event)
      return

    self._capture(log_event)
    self._log_header_if_necessary()
    self._log_row()

  def _capture(self, log_event):
    if log_event.in_train: 
      self.last_train_event = log_event
    else:
      self.last_valid_event = log_event
    
  def _log_row(self):
    if not self.last_train_event:
      return
    
    if not self.last_valid_event:
      return

    metrics_row = [str(self.epoch)] 
    for event in [self.last_train_event, self.last_valid_event]:
      for metric in event.avg_stats:
        metrics_row.append(f'{metric:.3f}')

    self.epochs_bar.write(metrics_row, table=True)
    self.last_train_event = False
    self.last_valid_event = False

  def _log_header_if_necessary(self):
    if self.logged_header:
      return
    self.logged_header = True
    metric_names = [m.__name__ for m in self.last_train_event.metrics]
    headers = ['epoch']
    headers += ['train_loss'] + [f'train_{n}' for n in metric_names]
    headers += ['valid_loss'] + [f'valid_{n}' for n in metric_names]
    self.epochs_bar.write(headers, table=True)

callback_funcs = [partial(AvgStatsCallback,accuracy),
        CudaCallback,
        ProgressCallbackV2,
        partial(BatchTransformXCallback, norm_imagenette)]

model_func = get_model_func()
loss_func = F.cross_entropy
num_filters = [32]*4
layer_func = conv_layer
lr = 0.1
learner = get_learner(num_filters, data, lr, layer_func, loss_func, cb_funcs=callback_funcs)
learner.fit(3)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy
0,2.020,0.275,1.800,0.394
1,1.669,0.435,1.472,0.526
2,1.455,0.509,1.379,0.542


In [0]:
# logger use to support the thing
# master pb & current pb
# two callback exchange data through a runner :/
# works for multiple iterations
# train_loss	train_accuracy	valid_loss	valid_accuracy	time

# Bottom